# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションにおける人間による好み予測のための機械学習モデルの構築に取り組んでいます。主な目標は、ユーザーのプロンプトに対する二つの異なるLLM（大規模言語モデル）が生成した応答のどちらが好まれるかを予測することです。

### 問題の概要
ノートブックでは、公開されているデータセットからトレーニングデータとテストデータを読み込み、それに基づいて予測を行うモデルを開発しています。

### 使用している手法とライブラリ
1. **データ処理**:
   - `pandas`や`datasets`ライブラリを用いて、データの読み込みや前処理を行っています。
   - テキストデータに対するクリーニング処理（小文字変換、URLやストップワードの削除など）を行う関数を定義しています。

2. **トークナイゼーション**:
   - `transformers`ライブラリのBERTモデルを使用して、テキストデータをトークン化しています。このプロセスでは、トークナイザーを初期化し、データセットへのトークン化を施しています。

3. **モデル構築**:
   - TensorFlowとKerasを利用し、カスタムBERTモデルを定義しています。`TFBertModel`を入れ子にしたクラスとして実装しており、データをBERTに通じて処理する層を作成しています。

4. **学習と評価**:
   - モデルは、`Adam`オプティマイザーを使用してコンパイルされ、エポックごとにトレーニングが行われます。訓練中の進捗は`TqdmCallback`を用いて表示されています。

5. **予測の生成**:
   - テストデータに対してモデルを用いて予測を実施し、その結果を`DataFrame`として整形してCSVファイルとして保存します。

ノートブック全体を通じて、高度な深層学習フレームワークや自然言語処理のテクニックが使用されており、特にBERTモデルの利用により、テキストデータに対して強力な表現学習を行なっています。

---


# 用語概説 
以下に、機械学習・深層学習の初心者がつまずきそうな専門用語の解説を示します。コンペティションのノートブックに特有の文脈を考慮し、あまり知られていない用語や実務経験がないと馴染みが薄いものに焦点を当てています。

1. **TPU（Tensor Processing Unit）**:
   - Googleが設計した、特に機械学習タスクに最適化されたハードウェア加速装置。従来のCPUやGPUよりも、高速に計算を行うことができる。Kaggleなどのプラットフォームで利用され、深層学習モデルのトレーニングを効率化する。

2. **Layer（レイヤー）**:
   - ニューラルネットワークの一部を構成する単位。入力データを受け取り、重みを掛け算し、活性化関数を通して出力を生成する。レイヤーは様々な種類があり、具体的な機能によって異なる（例えば、畳み込みレイヤー、全結合レイヤーなど）。

3. **Data Collator**:
   - トレーニングデータをバッチ処理する際にデータをまとめる役割を持つ。異なる長さの入力データを同じサイズにパディングし、一つのバッチにまとめる機能を提供する。

4. **attention_mask（アテンションマスク）**:
   - Transformerモデルにおいて、どのトークンに注目すべきかを示すマスク。特に、パディングされたトークンを無視するために使用される。これにより、無関係なトークンが計算に影響を与えないようにする。

5. **Sparse Categorical Crossentropy（疎形式のカテゴリカル交差エントロピー）**:
   - 多クラス分類問題において用いられる損失関数の一種。各クラスの正解ラベルが整数で表される場合に使用される。この損失関数は、モデルが予測した確率と実際のクラスとの違いを測る。

6. **Global Average Pooling（グローバル平均プーリング）**:
   - CNN（畳み込みニューラルネットワーク）などのモデルにおけるレイヤーで、特徴マップ全体の平均値を計算して出力とする方法。これにより、入力データの空間的な特徴を集約でき、多数のパラメータを減らすことができる。

7. **Tokenization（トークナイゼーション）**:
   - テキストデータを小さな単位（トークン）に分割するプロセス。自然言語処理において、単語や文を処理可能な形式にするために行われ、トークンはモデルの入力として使用される。

8. **BERT（Bidirectional Encoder Representations from Transformers）**:
   - 自然言語処理のための事前学習されたモデル。双方向的に文脈を捉えることができ、様々なNLPタスクで高い性能を発揮する。トークン化とエンコーディングの機能を備えており、このモデルを使用して特徴を抽出する。

9. **MirroredStrategy**:
   - TensorFlowの分散学習において、モデルを単一のマシン上の複数のGPUにミラーリングするための戦略。モデルのパラメータを各GPUに同期させ、トレーニングプロセスを並列化する。

10. **Gradient Descent（勾配降下法）**:
    - 機械学習モデルを最適化するために利用されるアルゴリズム。損失関数の勾配を計算し、その負の方向にパラメータを更新していくことで、最小値を見つけることを目的とする。最適化手法の一つで、最も基本的な形態の一つ。

これらの用語は、ノートブック内で使用される背景や概念の理解を深めるのに役立つでしょう。初心者がこれらの専門用語に遭遇した際に、スムーズに理解できるように解説しています。

---


In [ ]:
# osモジュールをインポートします。OS関連の機能を扱うために使用します。
import os
# tensorflowライブラリをインポートします。深層学習のフレームワークです。
import tensorflow as tf
# datasetsモジュールからload_datasetとDatasetDictをインポートします。データセットの読み込みと管理に使用します。
from datasets import load_dataset, DatasetDict
# transformersモジュールからモデルとトークナイザをインポートします。自然言語処理に用います。
from transformers import BertTokenizer, BertTokenizerFast, TFBertModel, DataCollatorWithPadding, TFAutoModel
# kerasのレイヤーをインポートします。ニューラルネットワークの構築に必要なレイヤーです。
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Lambda, Layer, Input, Dropout, GlobalAveragePooling2D
# kerasモデルをインポートします。モデルの構築と訓練に使います。
from tensorflow.keras.models import Model
# shutilモジュールをインポートします。ファイルやフォルダの操作に使用します。
import shutil
# pandasライブラリをインポートします。データの操作と解析用のライブラリです。
import pandas as pd
# tqdmのkerasコールバックをインポートします。進捗バーを表示するために使用します。
from tqdm.keras import TqdmCallback
# 正規表現を使うためのreモジュールをインポートします。
import re
# 数学関数を利用するためにmathモジュールをインポートします。
import math
# matplotlibのpyplotをインポートします。データの可視化に使用するグラフ描画ライブラリです。
import matplotlib.pyplot as plt
# multiprocessingモジュールをインポートします。マルチプロセッシングを扱うためのライブラリです。
import multiprocessing
# nltkライブラリをインポートします。自然言語処理に関連する機能を提供します。
import nltk
# nltkからストップワードをインポートします。テキスト処理で無視される単語のリストです。
from nltk.corpus import stopwords
# nltkから単語トークナイザをインポートします。文章を単語に分割するために使用します。
from nltk.tokenize import word_tokenize
# 学習率スケジューラのコールバックをインポートします。学習率を動的に変更するために使用します。
from tensorflow.keras.callbacks import LearningRateScheduler
# numpyライブラリをインポートします。数値計算のためのライブラリです。
import numpy as np
# kerasをインポートします。深層学習のためのライブラリです。
from tensorflow import keras
# Adam最適化アルゴリズムをインポートします。モデルの訓練で広く使用される最適化手法です。
from tensorflow.keras.optimizers import Adam

In [ ]:
# GPUの利用可能性をチェックします。
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPUが存在する場合、以下の処理を行います。
if gpus:
    try:
        # すべての物理GPUに対してメモリ成長を設定します。
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 論理GPUのリストを取得します。
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        # 物理GPUの数と論理GPUの数を表示します。
        print(len(gpus), "物理GPU,", len(logical_gpus), "論理GPU")
    # ランタイムエラーが発生した場合、そのエラーメッセージを表示します。
    except RuntimeError as e:
        print(e)

In [ ]:
# ハードウェアを検出し、適切な分散戦略を返します。
try:
    # TPUの検出を行います。TPU_NAME環境変数が設定されている場合はパラメータは必要ありません。
    # これはKaggle上では常に当てはまります。
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    # TPUのマスターアドレスを表示します。
    print('TPUで実行中: ', tpu.master())
# 値エラーが発生した場合、TPUをNoneに設定します。
except ValueError:
    tpu = None

# TPUが存在する場合
if tpu:
    # TPUクラスタに接続します。
    tf.config.experimental_connect_to_cluster(tpu)
    # TPUシステムを初期化します。
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # TPU用の分散戦略を設定します。
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # TensorFlowのデフォルトの分散戦略を使用します。これはCPUおよび単一GPUで機能します。
    strategy = tf.distribute.MirroredStrategy()

# 現在の戦略で同期しているレプリカの数を表示します。
print("レプリカの数: ", strategy.num_replicas_in_sync)

In [ ]:
# データセットのパスを設定します。
train_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'  # トレーニングデータのパス
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'    # テストデータのパス

# データセットを読み込みます。
# CSVファイルからトレーニングデータを読み込みます。
train_dataset = load_dataset('csv', data_files={'train': train_path})['train']
# CSVファイルからテストデータを読み込みます。
test_dataset = load_dataset('csv', data_files={'test': test_path})['test']

# テストデータセットからIDを保存します。
test_ids = test_dataset['id']  # テストデータのIDを取得します。

In [ ]:
# テストセットに不足している列を追加します。
for col in ['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie']:
    # 各列がテストデータセットに存在しない場合
    if col not in test_dataset.column_names:
        # 空の文字列で列を追加します。行数はテストデータセットの長さと同じです。
        test_dataset = test_dataset.add_column(col, [""] * len(test_dataset))

# 列のデータ型をint64に変換します。
for col in ['winner_model_a', 'winner_model_b', 'winner_tie']:
    # トレーニングデータセットに対して、指定した列を整数に変換します。
    train_dataset = train_dataset.map(lambda x: {col: int(x[col]) if x[col] is not None else 0})
    # テストデータセットに対して、指定した列を整数に変換します。
    test_dataset = test_dataset.map(lambda x: {col: int(x[col]) if x[col] != "" else 0})

In [ ]:
# ローカルでbert-base-casedのファイルを使用します。
source_dir = '/kaggle/input/huggingface-bert/bert-base-cased'  # ソースディレクトリのパス

# モデルの保存先ディレクトリを指定します。
model_dir = '/kaggle/working/bert-base-cased'
# 保存先ディレクトリが存在しない場合は作成します。
os.makedirs(model_dir, exist_ok=True)

# 必要なファイルをソースディレクトリからモデルディレクトリにコピーします。
shutil.copy(os.path.join(source_dir, 'config.json'), model_dir)  # 設定ファイルのコピー
shutil.copy(os.path.join(source_dir, 'pytorch_model.bin'), model_dir)  # PyTorchモデルファイルのコピー
shutil.copy(os.path.join(source_dir, 'tf_model.h5'), model_dir)  # TensorFlowモデルファイルのコピー
shutil.copy(os.path.join(source_dir, 'tokenizer.json'), model_dir)  # トークナイザー設定ファイルのコピー
shutil.copy(os.path.join(source_dir, 'vocab.txt'), model_dir)  # ボキャブラリファイルのコピー
shutil.copy(os.path.join(source_dir, 'modelcard.json'), model_dir)  # モデルカードのコピー

In [ ]:
# ストップワードのファイルパスを指定します。
stopwords_path = '/kaggle/input/stopwords/stopwords/english'

# ストップワードをファイルから読み込むための関数を定義します。
def load_stopwords(stopwords_path):
    # 指定したパスのファイルを開きます。
    with open(stopwords_path, 'r') as file:
        # ファイルの内容を読み込み、行ごとのリストに分割します。
        stopwords = file.read().splitlines()
    # ストップワードのリストをセットとして返します。集合にすることで重複を排除します。
    return set(stopwords)

In [ ]:
# トークナイザーを初期化します。
tokenizer = BertTokenizerFast.from_pretrained(model_dir)  # 事前学習済みのトークナイザーを指定したディレクトリから読み込みます。
# ストップワードをダウンロードします。
stopwords = load_stopwords(stopwords_path)  # ストップワードをファイルから読み込みます。

# テキストクリーニングのための関数を定義します。
def clean_text(text):
    # テキストを小文字に変換します。
    text = text.lower()
    # URLを削除します。
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # @メンションや#ハッシュタグを削除します。
    text = re.sub(r'\@\w+|\#','', text)
    # 句読点を削除します（英数字およびスペース以外の文字）。
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # ストップワードを削除します。テキストを単語に分割し、ストップワードに含まれない単語だけを結合します。
    text = ' '.join([word for word in text.split() if word not in stopwords]) 
    return text  # クリーンアップされたテキストを返します。

In [ ]:
def tokenize_function(examples):
    # 各テキストフィールドをクリーンアップします。
    cleaned_prompts = [clean_text(text) for text in examples['prompt']]  # プロンプトのクリーニング
    cleaned_responses_a = [clean_text(text) for text in examples['response_a']]  # 応答Aのクリーニング
    cleaned_responses_b = [clean_text(text) for text in examples['response_b']]  # 応答Bのクリーニング
    
    # クリーンアップされたテキストをトークン化します。
    return tokenizer(cleaned_prompts,
                     cleaned_responses_a,
                     cleaned_responses_b,
                     padding="max_length",  # 最大長さまでパディングします。
                     truncation=True,  # 最大長さを超えるテキストは切り捨てます。
                     max_length=512)  # 最大のトークン長を512に設定します。

In [ ]:
# 関数の使用例を示します。
examples = {
    'prompt': ["これはサンプルプロンプトです。"],  # サンプルプロンプト
    'response_a': ["これはサンプル応答Aです。"],  # サンプル応答A
    'response_b': ["これはサンプル応答Bです。"]   # サンプル応答B
}

# トークン化された出力を得ます。
tokenized_output = tokenize_function(examples)
# トークン化された出力を表示します。
print(tokenized_output)

In [ ]:
# マルチプロセッシングを使用してトークン化とクリーニング関数を適用します。num_procには使用するプロセッサの数を指定します。
num_proc = multiprocessing.cpu_count()  # 利用可能なCPUのコア数を取得します。

# より良いエラーハンドリングのためにtry-exceptブロックを追加します。
try:
    # トレーニングデータセットに対してトークン化関数を適用します。
    tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
    # テストデータセットに対してトークン化関数を適用します。
    test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True)
# エラーが発生した場合、そのエラーメッセージを表示します。
except Exception as e:
    print(f"トークン化中にエラーが発生しました: {e}")
    
# トークナイザーを使用して、パディングを行うためのデータコレータを作成します。
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# トークン化後にデバッグ用の印刷を追加します。
print("サンプルトークン化されたトレーニングデータセットのエントリー:")
print(tokenized_datasets[0])  # トークン化された最初のトレーニングデータセットのエントリーを表示します。
# トークン化されたトレーニングデータセットが空の場合、エラーを発生させます。
if len(tokenized_datasets) == 0:
    raise ValueError("トークン化されたトレーニングデータセットが空です。")
# トークン化されたテストデータセットが空の場合、エラーを発生させます。
if len(test_tokenized_datasets) == 0:
    raise ValueError("トークン化されたテストデータセットが空です。")
# デバッグ用にカラム名を印刷します。
print(f"トークン化されたトレーニングデータセットのカラム名: {tokenized_datasets.column_names}")
print(f"トークン化されたテストデータセットのカラム名: {test_tokenized_datasets.column_names}")

In [ ]:
# tf.data.Datasetに変換し、正しい形状にします。
def convert_to_tf_dataset(dataset, label_col=None, for_inference=False):
    input_columns = tokenizer.model_input_names  # トークナイザーのモデル入力名を取得します。
    
    # ラベル列が指定されており、推論用でない場合
    if label_col and not for_inference:
        # ラベル列を除く他の列を削除します。
        dataset = dataset.remove_columns([col for col in dataset.column_names if col != label_col and col not in input_columns])
    else:
        # インプット列に含まれない他の列を削除します。
        dataset = dataset.remove_columns([col for col in dataset.column_names if col not in input_columns])
    
    # ラベルがシーケンスでないことを確認します。
    if label_col:
        dataset = dataset.map(lambda x: {label_col: int(x[label_col])})  # ラベルを整数に変換します。
    
    shuffle = not for_inference  # 推論用でない場合はデータをシャッフルします。
    batch_size = 16 if for_inference else 450  # 推論用の場合はバッチサイズを16、それ以外は450に設定します。

    # データセットをtf.data.Datasetに変換します。
    tf_dataset = dataset.to_tf_dataset(
        columns=input_columns,  # 入力列
        label_cols=[label_col] if label_col and not for_inference else None,  # ラベル列
        shuffle=shuffle,  # シャッフルオプション
        batch_size=batch_size,  # バッチサイズ
        collate_fn=DataCollatorWithPadding(tokenizer=tokenizer)  # パディングを行うための関数
    )

    return tf_dataset  # 変換されたtf.data.Datasetを返します。

In [ ]:
# 変換を実行します。
try:
    # トークン化されたデータセットをtf.data.Dataset形式に変換します。ラベルとして'winner_model_a'を指定します。
    train_tf_dataset = convert_to_tf_dataset(tokenized_datasets, 'winner_model_a')
    # テストデータセットも同様に変換します。ここでもラベルとして'winner_model_a'を指定しますが、通常は異なるラベルを使用したほうが良いでしょう。
    test_tf_dataset = convert_to_tf_dataset(tokenized_datasets, 'winner_model_a')
# 変換中にエラーが発生した場合、そのエラーメッセージを表示します。
except Exception as e:
    print(f"データセットの変換中にエラーが発生しました: {e}")

In [ ]:
# データセットの変換後にデバッグ用の印刷を追加します。
print("変換されたトレーニングtf.data.Datasetのサンプル:")
# トレーニングデータセットから1つのバッチを取得します。
for batch in train_tf_dataset.take(1):
    inputs, labels = batch  # 入力とラベルを取得します。
    # 入力IDの形状を表示します。
    print(f'入力IDの形状: {inputs["input_ids"].shape}')
    # アテンションマスクの形状を表示します。
    print(f'アテンションマスクの形状: {inputs["attention_mask"].shape}')
    # ラベルの形状を表示します。
    print(f'ラベルの形状: {labels.shape}')

In [ ]:
# カスタムモデルを構築します。
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # 事前学習済みのBERTモデルを読み込みます。
        self.bert = TFBertModel.from_pretrained(model_dir, from_pt=True)
    
    def call(self, inputs):
        # 入力からinput_idsとattention_maskを取得します。
        input_ids, attention_mask = inputs
        # BERTモデルに入力を渡して出力を取得します。
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # BERTの最終隠れ状態を返します。

def create_keras_model():
    # 入力ID用の入力レイヤーを定義します。
    input_ids = Input(shape=(512,), dtype=tf.int32, name='input_ids')
    # アテンションマスク用の入力レイヤーを定義します。
    attention_mask = Input(shape=(512,), dtype=tf.int32, name='attention_mask')

    # BERT層を通して出力を取得します。
    bert_output = BertLayer()([input_ids, attention_mask])
    # 出力をグローバル平均プーリングします。
    pooled_output = GlobalAveragePooling1D()(bert_output)
    # 最終出力レイヤーを定義します。ここでは3つのクラス用にソフトマックス活性化関数を使用します。
    output = Dense(3, activation='softmax')(pooled_output)

    # モデルを構築します。入力と出力を指定します。
    model = Model(inputs=[input_ids, attention_mask], outputs=output)
    return model  # 作成したモデルを返します。

In [ ]:
# ストラテジースコープ内でモデルを構築し、コンパイルします。
with strategy.scope():
    # Kerasモデルを作成します。
    model = create_keras_model()
    # モデルをコンパイルします。最適化手法としてAdamを使用し、学習率は5e-5に設定します。
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                  loss='sparse_categorical_crossentropy',  # 損失関数には疎形式のカテゴリカル交差エントロピーを使用します。
                  metrics=['accuracy'])  # 評価指標には精度を使用します。

    # トレーニングデータセットでモデルを訓練します。エポック数は3です。
    model.fit(train_tf_dataset, epochs=3, callbacks=[TqdmCallback(verbose=1)])  # TqdmCallbackを使用して進捗を表示します。

In [ ]:
# 予測を取得します。
predictions = model.predict(test_tf_dataset)  # テストデータセットに対して予測を行います。

# 長さをチェックします。
print(f"テストIDの長さ: {len(test_ids)}")  # テストIDの数を表示します。
print(f"予測の形状: {predictions.shape}")  # 予測の形状を表示します。
# テストIDの数と予測の数が一致しない場合、予測をテストIDの数に合わせて調整します。
if len(test_ids) != predictions.shape[0]:
    predictions = predictions[:len(test_ids)]  # 余分な予測を切り捨てます。

# データフレームを作成します。
submission = pd.DataFrame({
    'id': test_ids,  # テストIDをID列として追加します。
    'winner_model_a': predictions[:, 0],  # モデルAの予測を列として追加します。
    'winner_model_b': predictions[:, 1],  # モデルBの予測を列として追加します。
    'winner_model_tie': predictions[:, 2]  # タイの予測を列として追加します。
})

# データフレームをCSVファイルとして保存します。
submission.to_csv('submission.csv', index=False)  # インデックスなしでCSVファイルに保存します。

# コメント

> ## ハッサン・シャヒディ
> 
> 素晴らしい仕事です！その調子でがんばってください🔥
> 
> 

---

> ## ギータ・ケオテ
> 
> よくやりました！その調子でがんばってください👍
> 
> 

---

> ## フー・フランシー
> 
> あなたの仕事が好きです！
> 
> 

---

> ## デッドQ
> 
> Kaggleコミュニティのための良いコードです。ありがとうございます！その調子でがんばってください！
> 
> 

---

> ## ヴァンシュ・シャルマ
> 
> [@fulgrimthe](https://www.kaggle.com/fulgrimthe) あなたの分析は明確で非常に役立ちます。素晴らしい仕事です！
> 
> 

---